In [31]:
import numpy as np
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
model_iMM904 = read_sbml_model('iMM904.xml')
phy = Metabolite(id='phy', name='phytoene', compartment='c') #Adding the metabolite
new_reaction1 = Reaction('phy_syn') #PHYtoene SYNtase
phy = Metabolite(id='phy', name='phytoene', compartment='c') #Adding the metabolite

new_reaction1.add_metabolites({model_iMM904.metabolites.ggdp_c: -2,
                              phy: 1, # our newly created metabolites
                               model_iMM904.metabolites.ppi_c: 2
                                })
model_iMM904.add_reactions([new_reaction1])
new_reaction2 = Reaction('neu_syn') #NEUrosporene SYNtase
#Adding the metabolite
neu = Metabolite(id='neu', name='Neurosporene', compartment='c') 
#AH2 = Metabolite(id='AH2', name='Reduced acceptor', compartment='c') 
#A = Metabolite(id='A', name='Acceptor', compartment='c') 
new_reaction2.add_metabolites({
    model_iMM904.metabolites.nadp_c: -3,
    phy: -1, # our newly created metabolites
    model_iMM904.metabolites.nadph_c: 3,
    neu: 1,
    })
model_iMM904.add_reactions([new_reaction2])
new_reaction3 = Reaction('lyc_syn') #LYCopene SYNtase
lyc = Metabolite(id='lyc', name='Lycopene', compartment='c') #Adding the metabolite
new_reaction3.add_metabolites({
    neu: -1, # our newly created metabolites
    model_iMM904.metabolites.nadph_c: -1, 
    model_iMM904.metabolites.o2_c: -1,
    lyc: 1,
    model_iMM904.metabolites.nadp_c: 1,
    model_iMM904.metabolites.h2o_c: 2,
    })
model_iMM904.add_reactions([new_reaction3])
new_reaction4 = Reaction('beta_syn') #BETA-carotene SYNtase
beta = Metabolite(id='beta', name='Beta-carotene', compartment='c', formula ='C40H56') #Adding the metabolite
new_reaction4.add_metabolites({
    lyc: -1, # our newly created metabolites
    beta: 1,
    })
model_iMM904.add_reactions([new_reaction4])
new_reaction5 = Reaction('EX_beta') #BETA-carotene SYNtase
new_reaction5.add_metabolites({
    beta: -1 #Excretion of beta-carotene
    })
model_iMM904.add_reactions([new_reaction5])
with model_iMM904:
    model_iMM904.objective = model_iMM904.reactions.EX_beta 
    print("Maximum beta-carotene flow: ",model_iMM904.optimize().objective_value)

Maximum beta-carotene flow:  0.35143769968051247


In [32]:
print(model_iMM904.metabolites.beta.elements)

{'C': 40, 'H': 56}


In [33]:
C = 12
H = 1
N = 14
O = 16
biomass_weight = C*47.5+H*7+N*9.25 #g/mol - this is the median measured
beta_weight = model_iMM904.metabolites.beta.elements['C']*C+model_iMM904.metabolites.beta.elements['H']*H #g/mol
gxtsn_e_weight = model_iMM904.metabolites.xtsn_e.elements['C']*C+model_iMM904.metabolites.xtsn_e.elements['H']*H+model_iMM904.metabolites.xtsn_e.elements['O']*O+model_iMM904.metabolites.xtsn_e.elements['N']*N
glucose_weight = model_iMM904.metabolites.glc__D_c.elements['C']*C+model_iMM904.metabolites.glc__D_c.elements['H']*H+model_iMM904.metabolites.glc__D_c.elements['O']*O
    

In [34]:
genes_for_kncokout = ['GAPD','PYK','ARGN','ARGN', 'PYRDC']

In [35]:
print(type(genes_for_kncokout[0]))

<class 'str'>


In [36]:
model_iMM904.reactions.get_by_id(genes_for_kncokout[0])

Reaction identifier,GAPD
Name,Glyceraldehyde-3-phosphate dehydrogenase
Memory address,0x07f9f19b2de20
Stoichiometry,g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c Glyceraldehyde 3-phosphate + Nicotinamide adenine dinucleotide + Phosphate <=> 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide - reduced
GPR,YJL052W or YJR009C or YGR192C
Lower bound,-999999.0
Upper bound,999999.0


In [46]:
genes_for_kncokout = ['GAPD','PYK','ARGN','ARGN', 'PYRDC']
for i in range(len(genes_for_kncokout)):
    with model_iMM904:
        print("\033[1m\nYield when", genes_for_kncokout[i], "is knockout out.\033[0m")
        model_iMM904.reactions.get_by_id(genes_for_kncokout[i]).bounds = 0,0
        model_iMM904.objective = model_iMM904.reactions.BIOMASS_SC5_notrace 
        solution = model_iMM904.optimize()
        max_growth_flux = model_iMM904.optimize().objective_value
        model_iMM904.reactions.BIOMASS_SC5_notrace.bounds = max_growth_flux*0.999, 100

        model_iMM904.objective = model_iMM904.reactions.EX_beta 
        solution_beta = model_iMM904.optimize()
        max_beta_flux = model_iMM904.optimize().objective_value 

        print('Investigation of maximum beta-carotene yield with a min growth flux of 99,9 % of the max groth flow:', round(max_growth_flux*0.999,2))
        yield_b_glc_g = (max_beta_flux /abs(solution_beta.fluxes["EX_glc__D_e"]))*(beta_weight/glucose_weight)
        yield_b_glc_mol = max_beta_flux /abs(solution_beta.fluxes["EX_glc__D_e"])
        yield_b_X_g = (max_beta_flux /abs(solution_beta.fluxes["BIOMASS_SC5_notrace"]))*(beta_weight/biomass_weight)
        yield_b_X_mol = max_beta_flux /abs(solution_beta.fluxes["BIOMASS_SC5_notrace"])

        #print 
        print()
        print("Max beta-carotene flux on glucose:",round(max_beta_flux,6))
        print("yield beta/glc:\t\t", round(yield_b_glc_mol,6), "\tmol/mol")
        print("Yield beta/glc:\t\t", round(yield_b_glc_g*1000,2), "\t\tmg/g\n")
        print("yield beta/x:\t\t", round(yield_b_X_mol,6), "\tmol/mol")
        print("Yield beta/x:\t\t", round(yield_b_X_g*1000,2), "\t\tmg/g")


Yield when GAPD is knockout out.
Investigation of maximum beta-carotene yield with a min growth flux of 99,9 % of the max groth flow: 0.04

Max beta-carotene flux on glucose: 6.3e-05
yield beta/glc:		 0.00011 	mol/mol
Yield beta/glc:		 0.33 		mg/g

yield beta/x:		 0.001604 	mol/mol
Yield beta/x:		 1.22 		mg/g

Yield when PYK is knockout out.
Investigation of maximum beta-carotene yield with a min growth flux of 99,9 % of the max groth flow: 0.05

Max beta-carotene flux on glucose: 0.000107
yield beta/glc:		 0.000163 	mol/mol
Yield beta/glc:		 0.49 		mg/g

yield beta/x:		 0.002036 	mol/mol
Yield beta/x:		 1.54 		mg/g

Yield when ARGN is knockout out.
Investigation of maximum beta-carotene yield with a min growth flux of 99,9 % of the max groth flow: 0.29

Max beta-carotene flux on glucose: 0.00061
yield beta/glc:		 6.1e-05 	mol/mol
Yield beta/glc:		 0.18 		mg/g

yield beta/x:		 0.00212 	mol/mol
Yield beta/x:		 1.61 		mg/g

Yield when ARGN is knockout out.
Investigation of maximum beta-